# Natrual Language Processing Project - Twitter Analysis

# Setting Enviroment

In [1]:
!pip install spacy-transformers gensim pyLDAvis statsmodels
!cp -f ./dictionary.py ~/venv/lib/python3.9/site-packages/gensim/corpora/dictionary.py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 KB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 111.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 110.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.7/380.7 KB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 KB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 924.9/924.9 KB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
#import gensim
import spacy
import tqdm
from nltk.tokenize import TweetTokenizer
import re
from typing import Iterable
import unicodedata
import torch
import statsmodels
import pyLDAvis.lda_model
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import plotly.express as px

#Sentiment analysis
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax
import torch


/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-27 09:28:42.286356: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 09:28:42.443786: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-27 09:28:42.443834: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-27 

In [3]:
import sys
try:
    spacy.load("en_core_web_md")
except OSError:   
    spacy.cli.download("en_core_web_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 49.8 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [4]:
nlp = spacy.load('en_core_web_md',disable=['parser','ner'])
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Data Import and Pre-processing

Data obtained Kaggle 500k ChatGPT-related Tweets

In [5]:
import pandas as pd
df_clean = pd.read_csv('twitter_jan_mar.csv') # Upload the data

In [6]:
df = df_clean.copy(deep=True) # We can use this to come back to a clean dataset

¿What kind of data are?

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500036 entries, 0 to 500035
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date           500036 non-null  object 
 1   id             500030 non-null  object 
 2   content        500030 non-null  object 
 3   username       500002 non-null  object 
 4   like_count     499974 non-null  float64
 5   retweet_count  499974 non-null  float64
dtypes: float64(2), object(4)
memory usage: 22.9+ MB


Search for null values and deleted them

In [8]:
df.isnull().sum()

date              0
id                6
content           6
username         34
like_count       62
retweet_count    62
dtype: int64

In [9]:
df = df.dropna() 

Convertion of date column from object to datetime

In [10]:
df.date.sample(5)

389741    2023-01-26 23:48:29+00:00
366076    2023-01-31 16:54:45+00:00
4542      2023-03-29 12:49:40+00:00
455103    2023-01-13 20:57:16+00:00
53968     2023-03-22 20:29:19+00:00
Name: date, dtype: object

In [11]:
# We add the timestamps to a daily level
df['date'] = pd.to_datetime(df['date']).dt.date 
df['date'] = pd.to_datetime(df['date'])         

In [12]:
print('Primera fecha:',df['date'].min())
print('Última fecha:',df['date'].max())

Primera fecha: 2023-01-04 00:00:00
Última fecha: 2023-03-29 00:00:00


In [13]:
def rm_punctuation(text:str) -> str:
    pattern = re.compile(r'[^a-zñ\s]', flags=re.IGNORECASE)
    return pattern.sub(" ",text)


def strip_accents(text:str) -> str:
    text_no_accents = ''.join(c for c in unicodedata.normalize('NFD', text)
                  if unicodedata.category(c) != 'Mn')        
    return text_no_accents


def pos_filter_and_lematize(
    text:str,
    allowed_postags:Iterable[str] = ("NOUN", "ADJ", "VERB", "ADV","PROPN")
) -> str:
    allowed_postags = set(allowed_postags)
    doc = nlp(text)

    tokens = [token.lemma_ for token in doc if token.pos_ in allowed_postags]
    return " ".join(tokens)


def preprocess_data(text:str) -> str:
    # Remove links
    clean_text = text
    clean_text = re.sub('http://\S+|https://\S+', '', clean_text)
    clean_text = re.sub('http[s]?://\S+', '', clean_text)
    clean_text = re.sub(r"http\S+", "", clean_text)

    clean_text = re.sub('&amp', 'and', clean_text)
    clean_text = re.sub('&lt', '<', clean_text)
    teclean_text = re.sub('&gt', '>', clean_text)

    # Remove hashtags
    clean_text = re.sub(r'#\w+', '', clean_text)

    clean_text = rm_punctuation(clean_text)
    clean_text = strip_accents(clean_text)

    # Remove handles and reduce words with more leters to increase an expresion 
    twt_tokenizer = TweetTokenizer(
        preserve_case=False, 
        strip_handles=True, 
        reduce_len=True
    )
    
    clean_text = ' '.join(twt_tokenizer.tokenize(clean_text))

    # Remove extra spaces
    clean_text = re.sub('\s+',' ', clean_text)

    # Lematize
    clean_text = pos_filter_and_lematize(clean_text)

    return clean_text

<>:26: DeprecationWarning: invalid escape sequence \S
<>:27: DeprecationWarning: invalid escape sequence \S
<>:51: DeprecationWarning: invalid escape sequence \s
<>:26: DeprecationWarning: invalid escape sequence \S
<>:27: DeprecationWarning: invalid escape sequence \S
<>:51: DeprecationWarning: invalid escape sequence \s
/tmp/ipykernel_122/1349833991.py:26: DeprecationWarning: invalid escape sequence \S
  clean_text = re.sub('http://\S+|https://\S+', '', clean_text)
/tmp/ipykernel_122/1349833991.py:27: DeprecationWarning: invalid escape sequence \S
  clean_text = re.sub('http[s]?://\S+', '', clean_text)
/tmp/ipykernel_122/1349833991.py:51: DeprecationWarning: invalid escape sequence \s
  clean_text = re.sub('\s+',' ', clean_text)


There are 500 thousand tweets in the set. We make a small chunk to work with less data and after seeing the result, work it with the rest.

In [14]:
df_small = df.sample(5000)
len(df_small)

5000

In [15]:
df_small['preprocessed_cont'] = df_small['content'].apply(preprocess_data)
df_small.head()

,date,id,content,username,like_count,retweet_count,preprocessed_cont
306561,2023-02-08,1623415766312968192,"#ChatGPT, do the DAN responses from people man...",Michael55698384,1.0,0.0,dan response people manipulate chatgpt indicat...
331917,2023-02-06,1622483254019497984,After months of stuffing around finally have a...,KiwiFunknuckle,26.0,0.0,month stuffing finally work phrase extraction ...
22812,2023-03-27,1640245296977387520,#TechTalk: How does ChatGPT work? - ZDNet\n\n#...,cichuck,0.0,0.0,chatgpt work zdnet read more here
157018,2023-03-07,1633116542224461825,Will #chatgpt transform the #insurance industr...,InsuranceBizUK,0.0,0.0,transform industry giasnape catch roi sproutai...
376216,2023-01-29,1619783331737964544,#AI &amp; #chatgpt will change #poetry for sur...,Climatius_,0.0,0.0,change sure change here come sonnet iambic pen...


In [16]:
df_small.preprocessed_cont.value_counts()[:10]

chat gpt                                                                                                               12
                                                                                                                        7
make top copywriter                                                                                                     6
themattmic chat gpt                                                                                                     4
join                                                                                                                    4
source                                                                                                                  4
sunrise sunset pm current temp c humidity wind speed km hr status clear generate base instruction                       3
use                                                                                                                     3
gpt vuln analyzer use ch

## Topic Modeling

In [17]:
# We add stopwords that we want to eliminate that appear with high frequency
stop_words = stop_words + ['chat', 'gpt', 'ai', 'chatgpt', 'use']

To make our theme model we need to create a Bag of Words

In [18]:
bow_vectorizer = CountVectorizer(
    
    stop_words=stop_words,
    min_df=1
)

X_bow_small = bow_vectorizer.fit_transform(df_small.preprocessed_cont)

In [19]:
lda_model = LatentDirichletAllocation(
    n_components=8,
    random_state=0,
    n_jobs=-1,
    verbose=1
)

lda_model.fit(X=X_bow_small)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(n_components=8, n_jobs=-1, random_state=0, verbose=1)

In [20]:
def show_topics_and_title(idx, lda_model, X_bow):
    topic_probs = lda_model.transform(X_bow[idx]).round(2)
    title = df_small.iloc[idx]['content']
    print(title)
    print(topic_probs)
    print(f"Topic más presente: {topic_probs.argmax()+1}")

for idx in range(10):
    show_topics_and_title(idx, lda_model, X_bow_small)
    print()

#ChatGPT, do the DAN responses from people manipulating ChatGPT indicate that it could potentially develop a multiple personality disorder?
[[0.01 0.01 0.28 0.01 0.01 0.01 0.66 0.01]]
Topic más presente: 7

After months of stuffing around finally have a working phrase extraction script thanks to #ChatGPT. Over 3 million phrases extracted and counted from around 10 am today.  Win!!!! https://t.co/d74606VpWs
[[0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.94]]
Topic más presente: 8

#TechTalk: How does ChatGPT work? - ZDNet

#AI #GPT #ML

Read More Here:
https://t.co/rES9FQFPaJ
[[0.03 0.03 0.78 0.03 0.03 0.03 0.03 0.03]]
Topic más presente: 3

Will #chatgpt transform the #insurance industry? @GiaSnape catches up with Roi Amir of @SproutAi and ​Amrit Santhirasenan of @hxtweets to find out at #InsurtechInsights - #insurancenews #insurtech   

https://t.co/flC9OQJCQK
[[0.01 0.01 0.92 0.01 0.01 0.01 0.01 0.01]]
Topic más presente: 3

#AI &amp; #chatgpt will change #poetry for sure, but how will it ch

In [21]:
pyLDAvis.enable_notebook()
bow_vectorizer.get_feature_names = bow_vectorizer.get_feature_names_out
LDAvis_prepared = pyLDAvis.lda_model.prepare(lda_model, X_bow_small, bow_vectorizer)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.019893  0.033600       1        1  16.054081
0      0.049165  0.026895       2        1  13.678628
2     -0.013488  0.071349       3        1  13.427575
1      0.044120 -0.103979       4        1  12.407147
7      0.037940 -0.003149       5        1  11.844931
6      0.009388  0.076954       6        1  11.391017
3      0.031931 -0.072258       7        1  10.717217
4     -0.178949 -0.029413       8        1  10.479405, topic_info=          Term        Freq       Total Category  logprob  loglift
3684    google  169.000000  169.000000  Default  30.0000  30.0000
786       bard   53.000000   53.000000  Default  29.0000  29.0000
7717    search  105.000000  105.000000  Default  28.0000  28.0000
5934       new  259.000000  259.000000  Default  27.0000  27.0000
4867      know  163.000000  163.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
4926  language   17.282438  128.869454   Topic8  -5.7242   0.2466
582        ask   21.329315  308.056050   Topic8  -5.5138  -0.4144
7645       say   17.348111  176.401407   Topic8  -5.7204  -0.0635
1438   chatbot   15.518475  115.570579   Topic8  -5.8319   0.2479
3676      good   15.459534  177.322576   Topic8  -5.8357  -0.1840

[661 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
14        2  0.790475  abacusai
44        5  0.802638  academia
55        1  0.459663    access
55        2  0.131332    access
55        3  0.218887    access
...     ...       ...       ...
9806      4  0.392044   youtube
9806      5  0.098011   youtube
9806      6  0.294033   youtube
9806      7  0.098011   youtube
9822      3  0.792127     zakir

[1731 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 3, 2, 8, 7, 4, 5])

## Exploratory Data Analysis (EDA)

### Likes vs Retweets 

In [22]:
# Define the maximum thresholds for like count and retweet count
max_like_count = 2000
max_retweet_count = 500

# Filter the DataFrame to include only rows where like_count and retweet_count are below the thresholds
df_filtered = df_small[(df_small['like_count'] <= max_like_count) & (df_small['retweet_count'] <= max_retweet_count)]


# Scatter plot of likes vs. retweets
fig = px.scatter(df_filtered, x='like_count', y='retweet_count', trendline="ols", trendline_color_override="red")
fig.update_layout(title='Likes vs. Retweets', xaxis_title='Likes', yaxis_title='Retweets', height=800, width=800)
fig.show()

### Tweets per day

In [23]:
sns.set_style('darkgrid')

# Number of tweets per day
tweets_by_day = df.groupby(pd.Grouper(key='date', freq='D')).size().reset_index()
tweets_by_day.columns = ['date', 'count']
fig2 = px.bar(tweets_by_day, x='date', y='count', title='Number of Tweets per Day', color = 'count', height=500, width=1000)
fig2.update_xaxes(tickangle=45, tickformat='%Y-%m-%d')
fig2.show()

### Tweets per month

In [24]:
tweets_by_month = df.groupby(pd.Grouper(key='date', freq='M',label='left')).size().reset_index(name='count')
fig = px.bar(tweets_by_month, x='date', y='count', title='Number of Tweets by Month', height=500, width=900, color = 'count')
fig.update_xaxes(title_text='Month')
fig.update_yaxes(title_text='Count')
fig.show()

## Sentiment Analysis with Roberta

In [25]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
resp=[]
for text in df_small.preprocessed_cont[:5]:
    thing=softmax(model(**tokenizer(text, return_tensors='pt'))[0][0].detach().numpy())
    resp.append((np.argsort(thing),thing))
resp

[(array([2, 1, 0]),
  array([0.75667053, 0.23321845, 0.01011099], dtype=float32)),
 (array([0, 1, 2]),
  array([0.00497664, 0.08862007, 0.90640324], dtype=float32)),
 (array([0, 2, 1]),
  array([0.02733884, 0.91541696, 0.05724419], dtype=float32)),
 (array([0, 2, 1]),
  array([0.0539801 , 0.85385066, 0.09216926], dtype=float32)),
 (array([0, 1, 2]),
  array([0.01398134, 0.44219947, 0.5438192 ], dtype=float32))]

In [27]:
#With this we try to force GPU usage.
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def sentiment(text):
    # Tokenize inputs
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Inference with PyTorch
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Convert logits to probabilities and sort
    probabilities = torch.softmax(logits, dim=1)
    probabilities = probabilities.cpu().numpy()[0]
    sorted_indices = np.argsort(probabilities)

    return sorted_indices, probabilities



Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
df_small.preprocessed_cont[:5].apply(sentiment)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


306561    ([2, 1, 0], [0.75667053, 0.23321845, 0.0101109...
331917    ([0, 1, 2], [0.0049766363, 0.08862007, 0.90640...
22812     ([0, 2, 1], [0.02733884, 0.91541696, 0.05724419])
157018    ([0, 2, 1], [0.053980105, 0.85385066, 0.092169...
376216    ([0, 1, 2], [0.013981337, 0.44219947, 0.5438192])
Name: preprocessed_cont, dtype: object

In [29]:
# Apply sentiment function to the preprocessed_cont column
sentiments = df_small.preprocessed_cont.apply(sentiment)
sentiments[:5]

306561    ([2, 1, 0], [0.75667053, 0.23321845, 0.0101109...
331917    ([0, 1, 2], [0.0049766363, 0.08862007, 0.90640...
22812     ([0, 2, 1], [0.02733884, 0.91541696, 0.05724419])
157018    ([0, 2, 1], [0.053980105, 0.85385066, 0.092169...
376216    ([0, 1, 2], [0.013981337, 0.44219947, 0.5438192])
Name: preprocessed_cont, dtype: object